## Kernel: stat_compound

In [ ]:
# uncomment if running in vs code
# %cd ..
# %cd ..

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
lat = 32.75
long = -79.75 + 360

df_u10 = pd.read_csv('Data/raw_data/u10_era5.csv')
df_v10 = pd.read_csv('Data/raw_data/v10_era5.csv')
df_msl = pd.read_csv('Data/raw_data/msl_era5.csv')
df_tp = pd.read_csv('Data/raw_data/tp_era5.csv')
df_met = pd.read_csv('Data/raw_data/Charleston_meteo.csv')
df_wl = pd.read_csv('Data/raw_data/Charleston_waterlevel.csv')
df_dis = pd.read_csv('Data/raw_data/USGS.csv')

In [4]:
def set_timezone(df, local, UTC, variable):
    '''
    Make time column a datetime, ensure time zone is UTC and set as index.
    Ambiguous and nonexistant times are set to Not a Time.
    Only returns a dataseries with variable and datetime as index at hourly intervals

    Parameters
    ----------
    variable: list of str or str
        Column name(s) to be indexed
    '''
    df = df.rename(columns={df.columns[0]: f'DateTime({UTC})'})
    df = df.set_index(pd.to_datetime(df.iloc[:, 0])) # time column should be the first column
    df = df.loc[:, variable]
    df = df.resample('h', origin = 'start').max() # resample to hourly and retain max
    df.index = df.index.tz_localize(
        local, ambiguous= 'NaT', nonexistent='NaT').tz_convert(UTC)
    # Remove duplicate rows based on the index
    df = df[~df.index.duplicated(keep='first')]
    return df

In [ ]:
# era 5 timezones are utc
df_u10 = set_timezone(df_u10, "UTC", "UTC", "u10")
df_u10.head()

In [ ]:
df_v10 = set_timezone(df_v10, "UTC", "UTC", "v10")
df_v10.head()

In [ ]:
df_tp = set_timezone(df_tp, "UTC", "UTC", "tp") * 1000 # m / time step to mm / timestep
df_tp.head()

In [ ]:
df_msl = set_timezone(df_msl, "UTC", "UTC", "msl")
df_msl.head()

In [ ]:
df_dis = set_timezone(df_dis, "US/Eastern", "UTC", "Discharge (metric)")
df_dis.head()

In [ ]:
df_wl = set_timezone(df_wl, "US/Eastern", "UTC", [df_wl.columns[-2], df_wl.columns[-1]])
df_wl.head()

Applying 1 year mean moving average to non tidal residual to remove trend in mean sea level. Applying it to the NTR prevents averaging out longer tidal cycles that can occur with a period that is longer than a year. This average is then remove from the total water level.

In [ ]:
wl_coincide = df_wl.dropna().loc['1922':'2023']
non_tidal_res = wl_coincide.iloc[:, 0] - wl_coincide.iloc[:, 1]
mov_year = non_tidal_res.rolling(24*365, min_periods = 24*7*4*6).mean()
mov_year.plot()
plt.title('One Year Moving Average of Non-Tidal Residual')
plt.ylabel('Watler Level [m]')
plt.grid()

In [ ]:
wl_coincide['WL_detrended (m)'] = wl_coincide.iloc[:, 0] - mov_year
wl_coincide['WL_trend (m)'] = mov_year
wl_coincide.head() # NaN's because minimum period to perform the moving average is six months. Not a problem as no other data available in 1922

In [ ]:
df_met = set_timezone(df_met, "US/Eastern", "UTC", [
                                                    df_met.columns[-3],
                                                    df_met.columns[-2],
                                                    df_met.columns[-1]
                                                    ])
df_met

In [14]:
mag = df_met[df_met.columns[-1]].values
direc = np.pi/180 * (df_met[df_met.columns[-2]].values - 180) # convert so that direction is now "to" not "from"
u_x = mag * np.sin(direc)
u_y = mag * np.cos(direc)
df_met['u10'] = u_x
df_met['v10'] = u_y
df_met['msl'] = df_met[df_met.columns[-3]].values * 100 # convert mb to pa for pressure
df_noaa_met = df_met['2022-01-01':].loc[:, ['u10', 'v10', 'msl']]

In [15]:
df_era_2021 = pd.concat([df_u10, df_v10, df_msl], 
    axis = 1, ignore_index = False)[:'2021-12-31']

In [ ]:
# To extend time record by 2 years append data gathered by NOAA at Charleston gauge
df_merged_met = pd.concat([df_era_2021, df_noaa_met], axis=0)
df_merged_met

In [ ]:
datum = []
for col in wl_coincide.columns:
    par = col.find('(')
    par_2 = col.find(')')
    datum.append('(' + col[par+1:par_2] + ')')
datum

In [18]:
df_all = pd.concat([df_merged_met, df_tp, df_dis, wl_coincide],
    axis = 1, ignore_index = False).rename(columns={'u10': 'hor_wind (m/s)',
                                                    'v10': 'ver_wind (m/s)',
                                                    'msl': 'Pressure (pa)',
                                                    'tp': 'Precipitation (mm/hr)',
                                                    'Discharge (metric)': 'Q (m^3/s)',
                                                    wl_coincide.columns[0]: 'WL ' + datum[0],
                                                    wl_coincide.columns[1]: 'Tidal (m)',
                                                    wl_coincide.columns[2]: 'WL_detrend (m)'})

In [19]:
df_all = df_all.reset_index().sort_values(by = 'DateTime(UTC)')
df_all = df_all.set_index((df_all.iloc[:, 0])).drop(columns = 'DateTime(UTC)')
# high limit, because discharge is missing data in 2001!
# remove constant discharge at start of record
df_record = df_all.bfill(limit = 365 * 24).dropna().iloc[364 * 24:]

In [20]:
diffs = df_record.index.to_series().diff()[1:]
indices_with_non_hourly_delta = diffs[diffs != pd.Timedelta(hours=1)].index
if len(indices_with_non_hourly_delta) != 0:
    print(indices_with_non_hourly_delta)
    raise Exception('Missing data, check limit when filling in NaN values')


In [ ]:
# hardcorded to ensure that there the number of days in the record is an integer
df_record = df_record.iloc[2:]
df_record

Remove MMSL which could be caused by atmospheric pattern on a monthly-yearly scale, skewing durations and magnitudes of skew surge

In [22]:
years = df_record.index.year.unique()
months = np.arange(1, 13, 1)
mmsl_array = np.zeros((len(years), len(months)))
for year in years:
    for month in months:
        try:
            wl_month_i = df_record.loc[str(year) + '-' + str(month), 'WL_detrend (m)'].copy()
            mmsl = wl_month_i.mean()
            mmsl_array[year - years[0], month - 1] = mmsl
            df_record.loc[str(year) + '-' + str(month), 'WL_detrend_nommsl (m)'] = wl_month_i.values - mmsl
        except KeyError:
            pass


In [ ]:
import matplotlib.pyplot as plt
plt.boxplot(mmsl_array)
plt.title('Seasonality in Sea Level Throughout the Year')
plt.xlabel('Months of the Year')
plt.ylabel('Monthly Mean Sea Level [m]')
label = ['Jan', 'Feb', 'Mar',
         'Apr', 'May', 'Jun',
         'Jul', 'Aug', 'Sep',
         'Oct', 'Nov', 'Dec']
plt.xticks(months, label)
plt.ylim([-0.3, 0.3])
plt.grid()
plt.show()

Tidal cycle defined as trough to trough

In [ ]:
from scipy.signal import argrelextrema

troughs = argrelextrema(df_record['Tidal (m)'].values, np.less)
troughs = troughs[0]
troughs

In [25]:
skew_surge_list = []
peak_tides = []
tidal_seasonal = df_record['Tidal (m)']

under_1 = 0
for i in range(len(troughs) + 1):
    if i != len(troughs):
        under_2 = troughs[i]
    else:
        under_2 = -1
       
    tidal_cycle_surge = df_record.iloc[under_1:under_2, [-4, -3]]
    peak_tide = tidal_cycle_surge.iloc[:, 0].values.max()
    peak_wl = tidal_cycle_surge.iloc[:, 1].values.max()
    skew_surge = peak_wl - peak_tide
    skew_surge_list.append(skew_surge)
    
    peak_tide = tidal_seasonal.iloc[under_1:under_2].values.max()
    peak_tides.append(peak_tide)
    
    under_1 = under_2

In [ ]:
df_record['Skew_surge (m)'] = np.nan
df_record.iloc[troughs, -1] = skew_surge_list[1:]
df_record.iloc[0, -1] = skew_surge_list[0]
df_record

In [27]:
df_record.to_csv('Data/all_drivers.csv', index = True)

In [ ]:
df_record.max()

In [ ]:
df_wl_ss = df_record.copy()
col_names = df_wl_ss.columns
df_wl_ss.drop(columns = [col_names[0], col_names[1], col_names[2], col_names[4], col_names[5]], inplace = True)
df_wl_ss.head()

In [ ]:
tidal_peaks = df_wl_ss['Skew_surge (m)'].dropna().to_frame()
tidal_peaks['High Tide'] = peak_tides
tidal_peaks['Diff'] = np.insert(np.diff(tidal_peaks.loc[:, 'High Tide']), 0, np.nan)
if tidal_peaks['Diff'].iloc[1] > 0:
    base = 'LH'
    other = 'HH'
else:
    base = 'HH'
    other = 'LH'
tidal_peaks['Type'] = base
tidal_peaks.loc[tidal_peaks['Diff'] > 0, 'Type'] = other

tidal_peaks

In [31]:
tidal_peaks.to_csv('Data/skew_surge_tides.csv')